## IMPORTS

In [1]:
%pip install sklearn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [136]:
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from skimage.io import imread, imshow
from skimage import feature
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image,ImageOps,ImageFile
import nltk
from nltk import word_tokenize,Text
from nltk.corpus import stopwords
import re
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## LOADING DATA

In [138]:
df = pd.read_pickle(r'/content/drive/MyDrive/Data/memotion_dataset_7k/labels_pd_pickle')

## IMAGE CLEANING

In [139]:
images=[]
for i in range(len(df['image_name'])):
    path = '/content/drive/MyDrive/Data/memotion_dataset_7k/images/'
    img=imread(path+df['image_name'].iloc[i],as_gray=True)
    if(img.shape[0]>2000):
        df['image_name'].iloc[i]=np.nan

/usr/local/lib/python3.7/dist-packages/skimage/io/_io.py:61: FutureWarning: Non RGB image conversion is now deprecated. For RGBA images, please use rgb2gray(rgba2rgb(rgb)) instead. In version 0.19, a ValueError will be raised if input image last dimension length is not 3.
  img = rgb2gray(img)
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


In [140]:
df.drop(df[df['text_corrected'].isnull()].index, inplace = True)
df.drop(df[df['image_name'].isnull()].index, inplace = True)
df

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...
6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive
6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


## OCR ACCURACY

In [141]:
# OCR TEXT 
print(len(df))
a= [1 for i in range (len(df))]
b= [1 if df['text_corrected'].iloc[i] == df['text_ocr'].iloc[i] else 0 for i in range(len(df))]
print("ocr accuracy is",accuracy_score(a,b)*100)

6968
ocr accuracy is 66.36050516647532


## TEXT CLEANING

In [142]:
sentiment={"very_positive":1,"positive":1,"neutral":0,"negative":2,"very_negative":2} #creating a dictionary to map the order of the values in column
motivational={"motivational":1,"not_motivational":0} 
sarcasm={"very_twisted":1,"twisted_meaning":1,"general":1,"not_sarcastic":0} 
humour={"hilarious":1,"very_funny":1,"funny":1,"not_funny":0} 
offense={"very_offensive":1,"slight":1,"not_offensive":0} 

In [143]:
X=df['text_corrected']

In [144]:
print(len(df),len(X))

6968 6968


In [145]:
df.reset_index()

,index,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...
6963,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6964,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6965,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive
6966,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


In [146]:
df['overall_sentiment']=df['overall_sentiment'].map(sentiment)
df['motivational']=df['motivational'].map(motivational)
df['sarcasm']=df['sarcasm'].map(sarcasm)
df['humour']=df['humour'].map(humour)
df['offensive']=df['offensive'].map(offense)


In [147]:
listss=[]
for i in df['overall_sentiment']:
  listss.append(i)
listss=np.array(listss)
lables=torch.tensor(listss)

In [148]:
print(len(X))

6968


In [149]:
sentiment=np.array(list(df['overall_sentiment']),dtype='float64')
humour=np.array(list(df['humour']),dtype='float64')
sarcasm=np.array(list(df['sarcasm']),dtype='float64')
motivational=np.array(list(df['motivational']),dtype='float64')
offensive=np.array(list(df['offensive']),dtype='float64')


In [150]:
X.fillna(" ", inplace=True)
x=X.tolist()
print(len(x))

6968


In [151]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [152]:
for i in range (0,len(x)):
    
    x[i]=x[i].lower()
    tokens=word_tokenize(x[i])
    for k in range(len(tokens)):
        tokens[k]= re.sub('[^A-Za-z0-9]+', '', tokens[k])    
    stop_words = set(stopwords.words('english'))
    filtered_sentence = [w for w in tokens if not w in stop_words]
    x[i]=""
    for j in filtered_sentence:
        x[i] += j + ' '
print(i)

6967


In [153]:
for i in range(len(x)):
    X.iloc[i]=x[i]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [154]:
X

0       look friend lightyear sohalikut trend play 10 ...
1       best  10 yearchallenge  completed less 4 years...
2       sam thorne  strippin  follow follow saw everyo...
3                   10 year challenge  sweet dee edition 
4       10 year challenge filter 47 hilarious 10 year ...
                              ...                        
6987    tuesday mardi gras wednesday valentine friday ...
6988    must watch movies 2017 iti chennai memes maana...
6989    less talking planning soda junk food complaini...
6990    time fantasy  one time unless make time  arhti...
6991    starting point every good idea    arhtistic li...
Name: text_corrected, Length: 6968, dtype: object

## EMBEDDINGS

In [155]:
from torchtext.vocab import GloVe
global_vectors = GloVe(name='6B', dim=100)
File = "/content/.vector_cache/glove.6B.100d.txt"
def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r',encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model 
embeddings=load_glove_model(File)
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch4


Loading Glove Model
400000 words loaded!


In [156]:
def find_max(t):
  max=0
  for i in t:
    print(i)
    if(len(i)>max):
      max=len(i)
  return max

In [157]:
find_max(df['text_corrected'])

Streaming output truncated to the last 5000 lines.
donald trump animatronic looks like started making one hillary switched donald halfway 
problem nt much hillary corrupt lying criminal  everyone knows  problem supporters nt care  
afraid army lions led centipede  afraid army centipedes led lion  american proverb 
shut german hillary slap canadian girlfriend 
news  hanne sus ws team eam hillary still somehow prison memescom 
trash treasonous reckless arrogant scandalprone hillary memefulcom 
ony fdny  nt remember seeing hillary clinton  ground zero 911   rudy giuliani trump rally look left rudy  
voting hillary  memecentercom must misogynist working patriarchy  
first debate hillary clinton 
textual relations server 
follow deaux  dstfelix  fix national epidemic hateration dancerie   
wants keep track guns  cant even keep track husband dick  irmgflipcom 
hillary prez heartless sociopath would nice change pace memescom 
hillary 2016 cause payin us vote memeshappencom 
weiner wife sends 

684

In [158]:
def pad_embeddings(emb,a):
    p=[]
    embedded=np.empty(0)
    a=a.lower()
    n=tokenizer(a)
    while(len(n)<684):
      n.append(' ')
    for i in n:
        try:
            m=emb[i].ravel()
            while(len(m)<100):
                m=np.append(m,0.0)
        except:
            m=np.zeros(100)        
        p.append(m)
    return p      


In [159]:
n=pad_embeddings(embeddings,df['text_corrected'].iloc[0])

### TENSOR CONVERSION

In [160]:
sent=torch.tensor(sentiment).float()
hmr=torch.tensor(humour).float()
motiv=torch.tensor(motivational).float()
off=torch.tensor(offensive).float()
sarc=torch.tensor(sarcasm).float()

In [186]:
images=list(df['image_name'])

In [187]:
path = '/content/drive/MyDrive/Data/memotion_dataset_7k/images/'

## Architecture

In [203]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.TextLayers = nn.Sequential(
            nn.Linear(684*100,512),
            nn.ReLU(),
            nn.Linear(512, 4096),
            nn.ReLU(),
            nn.Linear(4096, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
        )
        self.ImageLayers = nn.Sequential(
            nn.Linear(256*256,1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
        )
        
        
        self.Sentiment = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 3),
        )
        self.Motivation = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )
        self.Sarcasm = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )
        self.Offensive = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )
        self.Humourous = nn.Sequential(
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, image, text):
        text_out = self.TextLayers(text)
        image_out = self.ImageLayers(image)
        combined = torch.cat([image_out,text_out],0)
        sent = self.Sentiment(combined)
        humour = self.Humourous(combined)
        motiv = self.Motivation(combined)
        off = self.Offensive(combined)
        sarc = self.Sarcasm(combined)
        return sent,humour,motiv,off,sarc

In [204]:
network=NeuralNetwork()

In [205]:
criterion = nn.L1Loss()
optimizer = optim.Adam(network.parameters(), lr=0.00001)
loss_list=[]

In [206]:
x=df['text_corrected'].values

## TRAINING

In [210]:
for epoch in range(12):  # loop over the dataset multiple times
  print("Epoch ", epoch)
  running_loss_sentiment = 0.0
  running_loss_humour = 0.0
  running_loss_motivation = 0.0
  running_loss_sarcasm = 0.0
  running_loss_offense = 0.0
  for i in range(len(off)):
      # get the inputs; data is a list of [inputs, labels]
      img,text,sentiment,offensive,sarcasm,motivational,humour_ = images[i],pad_embeddings(embeddings,x[i]),sent[i],off[i],sarc[i],motiv[i],hmr[i]
      img=torch.tensor(resize(imread(path+img,as_gray=True),[256,256]).flatten()).float()
      # zero the parameter gradients
      optimizer.zero_grad()
      text=torch.tensor(np.array(text).flatten()).float()
      # forward + backward + optimize
      sent1,humour1,motiv1,off1,sarc1 = network(img.unsqueeze(0),text.unsqueeze(0))
      loss_sentiment = criterion(sent1, sentiment)
      loss_humour = criterion(humour1, humour_)
      loss_motivation = criterion(motiv1, motivational)
      loss_offense = criterion(off1, offensive)
      loss_sarcasm = criterion(sarc1, sarcasm)
      loss=loss_sentiment+loss_humour+loss_motivation+loss_offense+loss_sarcasm
      optimizer.step()

      # print statistics
      running_loss_sentiment += loss_sentiment.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss Sentiment: {running_loss_sentiment / 2000:.3f}')
        running_loss_sentiment = 0.0
      running_loss_humour += loss_humour.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss Humour: {running_loss_humour / 2000:.3f}')
        running_loss_humour = 0.0
      running_loss_motivation += loss_motivation.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss Motivation: {running_loss_motivation / 2000:.3f}')
        running_loss_motivation = 0.0
      running_loss_offense += loss_offense.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss Offense: {running_loss_offense / 2000:.3f}')
        running_loss_offense = 0.0
      running_loss_sarcasm += loss_sarcasm.item()
      if i % 2000 == 1999:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss Sarcasm: {running_loss_offense / 2000:.3f}')
        running_loss_sarcasm = 0.0
print('Finished Training')

Epoch  0


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([2, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


[1,  2000] loss Sentiment: 0.715
[1,  2000] loss Humour: 0.753
[1,  2000] loss Motivation: 0.403
[1,  2000] loss Offense: nan
[1,  2000] loss Sarcasm: 0.000
[1,  4000] loss Sentiment: 0.742
[1,  4000] loss Humour: 0.730
[1,  4000] loss Motivation: 0.419
[1,  4000] loss Offense: nan
[1,  4000] loss Sarcasm: 0.000


/usr/local/lib/python3.7/dist-packages/skimage/io/_io.py:61: FutureWarning: Non RGB image conversion is now deprecated. For RGBA images, please use rgb2gray(rgba2rgb(rgb)) instead. In version 0.19, a ValueError will be raised if input image last dimension length is not 3.
  img = rgb2gray(img)


[1,  6000] loss Sentiment: 0.733
[1,  6000] loss Humour: 0.735
[1,  6000] loss Motivation: 0.419
[1,  6000] loss Offense: nan
[1,  6000] loss Sarcasm: 0.000


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Epoch  1
[2,  2000] loss Sentiment: 0.715
[2,  2000] loss Humour: 0.753
[2,  2000] loss Motivation: 0.403
[2,  2000] loss Offense: nan
[2,  2000] loss Sarcasm: 0.000
[2,  4000] loss Sentiment: 0.742
[2,  4000] loss Humour: 0.730
[2,  4000] loss Motivation: 0.419
[2,  4000] loss Offense: nan
[2,  4000] loss Sarcasm: 0.000
[2,  6000] loss Sentiment: 0.733
[2,  6000] loss Humour: 0.735
[2,  6000] loss Motivation: 0.419
[2,  6000] loss Offense: nan
[2,  6000] loss Sarcasm: 0.000
Epoch  2
[3,  2000] loss Sentiment: 0.715
[3,  2000] loss Humour: 0.753
[3,  2000] loss Motivation: 0.403
[3,  2000] loss Offense: nan
[3,  2000] loss Sarcasm: 0.000
[3,  4000] loss Sentiment: 0.742
[3,  4000] loss Humour: 0.730
[3,  4000] loss Motivation: 0.419
[3,  4000] loss Offense: nan
[3,  4000] loss Sarcasm: 0.000
[3,  6000] loss Sentiment: 0.733
[3,  6000] loss Humour: 0.735
[3,  6000] loss Motivation: 0.419
[3,  6000] loss Offense: nan
[3,  6000] loss Sarcasm: 0.000
Epoch  3
[4,  2000] loss Sentiment: 0.71

### Saving Model

In [211]:
PATH = r'/content/drive/MyDrive/Data/model.pt'
torch.save(network.state_dict(), PATH)


In [216]:
%pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master
from torchviz import make_dot, make_dot_from_trace
from torchviz import make_dot
i=0
img,text,sentiment,offensive,sarcasm,motivational,humour_ = images[i],pad_embeddings(embeddings,x[i]),sent[i],off[i],sarc[i],motiv[i],hmr[i]
img=torch.tensor(resize(imread(path+img,as_gray=True),[256,256]).flatten()).float()
# zero the parameter gradients
optimizer.zero_grad()
text=torch.tensor(np.array(text).flatten()).float()
dot=make_dot(network(img.unsqueeze(0),text.unsqueeze(0)), params=dict(network.named_parameters()),show_attrs=True, show_saved=True)
dot.format = 'png'
dot.render("model")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/szagoruyko/pytorchviz.git (to revision master) to /tmp/pip-req-build-3xvnvtlk
  Running command git clone -q https://github.com/szagoruyko/pytorchviz.git /tmp/pip-req-build-3xvnvtlk


'model.png'